In [87]:
# importing neccessery libriries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import holidays

#folium and google collab
import folium
#import google.colab

In [88]:
#Load in datasets

df_spring = pd.read_csv("Data_raw/202404-capitalbikeshare-tripdata/202004-capitalbikeshare-tripdata.csv",sep = ",")

df_fall = pd.read_csv("Data_raw/202409-capitalbikeshare-tripdata/202409-capitalbikeshare-tripdata.csv",sep = ",")

In [89]:
df_spring.head(2)

#ride_id
#rideable_type
#started_at
#ended_at

#start_station_name
#start_station_id
#end_station_name
#end_station_id

#start_lat
#start_lng
#end_lat
#end_lng

#member_casual

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0DCF75B9433427B1,classic_bike,2024-04-29 17:03:15,2024-04-29 17:10:14,Jones Branch & Westbranch Dr,32206.0,McLean Metro,32230.0,38.931911,-77.219261,38.924002,-77.208127,member
1,EBA4FE55DA3CE4D2,classic_bike,2024-04-19 17:36:14,2024-04-19 17:41:31,14th & Otis Pl NW,31131.0,Lamont & Mt Pleasant NW,31107.0,38.934405,-77.032687,38.931900,-77.038800,member


In [90]:
df_spring.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id      float64
end_station_name       object
end_station_id        float64
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [91]:
df_spring['started_at'] = pd.to_datetime(df_spring['started_at'])
df_spring['ended_at'] = pd.to_datetime(df_spring['ended_at'])

df_fall['started_at'] = pd.to_datetime(df_fall['started_at'])
df_fall['ended_at'] = pd.to_datetime(df_fall['ended_at'])


In [92]:
df_fall.isnull().sum()
df_fall_nonull = df_fall.dropna()
df_spring_nonull = df_spring.dropna()




In [93]:
df_fall_nonull.isnull().sum()
df_spring_nonull.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [94]:
df_spring_nonull.head(2)
df_spring_nonull.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id             float64
end_station_name              object
end_station_id               float64
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

In [95]:


df_spring_daytime = df_spring_nonull[(df_spring_nonull['started_at'].dt.hour >= 8) 
        & (df_spring_nonull['started_at'].dt.hour <= 19)]  # filter by working hours



df_spring_daytime.shape

(308166, 13)

In [96]:
#Filtering dfs for morning rush/afternoon rush hours
df_spring_morning = df_spring_nonull[(df_spring_nonull['started_at'].dt.hour >= 8) 
        & (df_spring_nonull['started_at'].dt.hour <= 12)]  # filter by working hours

df_fall_morning = df_fall_nonull[(df_fall_nonull['started_at'].dt.hour >= 8) 
        & (df_fall_nonull['started_at'].dt.hour <= 12)]  # filter by working hours

df_spring_evening = df_spring_nonull[(df_spring_nonull['started_at'].dt.hour >= 13) 
        & (df_spring_nonull['started_at'].dt.hour <= 19)]  # filter by working hours

df_fall_evening = df_fall_nonull[(df_fall_nonull['started_at'].dt.hour >= 13) 
        & (df_fall_nonull['started_at'].dt.hour <= 19)]  # filter by working hours

In [97]:
df_spring_morning.shape

(101701, 13)

In [98]:
df_spring_morning.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [99]:
#Filter data frames to cut down on iteration times

df_spring_morning_filtered = df_spring_morning[["start_lat","start_lng","end_lat","end_lng"]]
df_spring_evening_filtered = df_spring_evening[["start_lat","start_lng","end_lat","end_lng"]]

df_fall_morning_filtered = df_fall_morning[["start_lat","start_lng","end_lat","end_lng"]]
df_fall_evening_filtered = df_fall_evening[["start_lat","start_lng","end_lat","end_lng"]]

df_spring_morning.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id             float64
end_station_name              object
end_station_id               float64
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

In [100]:
from folium.plugins import HeatMap

In [101]:

# Create a Folium map with a base layer
m = folium.Map(location=[38.889805,-77.009056], zoom_start=11) #starting lat lng of washington DC

# Define the HeatMap object with the start/end locations and frequency data
heat_data = [(row['start_lat'], row['start_lng']) for index, row in df_spring_morning_filtered.iterrows()]
heat_data += [(row['end_lat'], row['end_lng']) for index, row in df_spring_morning_filtered.iterrows()]

# Create the HeatMap
heatmap = folium.plugins.HeatMap(heat_data, radius=10)

# Add the HeatMap to the Folium map
heatmap.add_to(m)


monuments = {"name": ["United States Capitol","The White House","NoMa (North of Massachusetts Avenue)",
                      "The District of Columbia Public Library","Geography of Washington, D.C. (Zero Milestone on The Ellipse)"],
             "Lat": ["38.889805","38.897957","38.900497","38.942142","38.8903"],
             "Lng": ["-77.009056","-77.036560","-77.007507","-77.025955","-77.0352"],
             }
df_monuments_geo = pd.DataFrame.from_dict(monuments, orient='columns')


for index, row in df_monuments_geo.iterrows():
    lat, lon = row["Lat"], row["Lng"]
    # Create a marker with a custom icon and pop-up
    marker = folium.Marker([lat, lon], icon=folium.Icon(color='red'), popup=row['name'])
    # popup=[folium.Popup("START", parse_html=True, max_width=100),row['start_station_name']
    # Add the marker to the map
    m.add_child(marker)



# Save the map as an HTML file
m.save('spring_morning.html')

In [102]:
#COMMAND LINE
#python -m http.server 8000


#NAVIGATE TO :
#http://localhost:8000/travel_heatmap.html

In [103]:

# Create a Folium map with a base layer
m = folium.Map(location=[38.889805,-77.009056], zoom_start=11, min_opacity=0.2) #starting lat lng of washington DC

# Define the HeatMap object with the start/end locations and frequency data
heat_data = [(row['start_lat'], row['start_lng']) for index, row in df_spring_evening_filtered.iterrows()]
heat_data += [(row['end_lat'], row['end_lng']) for index, row in df_spring_evening_filtered.iterrows()]

# Create the HeatMap
heatmap = folium.plugins.HeatMap(heat_data, radius=10)

# Add the HeatMap to the Folium map
heatmap.add_to(m)



monuments = {"name": ["United States Capitol","The White House","NoMa (North of Massachusetts Avenue)",
                      "The District of Columbia Public Library","Geography of Washington, D.C. (Zero Milestone on The Ellipse)"],
             "Lat": ["38.889805","38.897957","38.900497","38.942142","38.8903"],
             "Lng": ["-77.009056","-77.036560","-77.007507","-77.025955","-77.0352"],
             }
df_monuments_geo = pd.DataFrame.from_dict(monuments, orient='columns')


for index, row in df_monuments_geo.iterrows():
    lat, lon = row["Lat"], row["Lng"]
    # Create a marker with a custom icon and pop-up
    marker = folium.Marker([lat, lon], icon=folium.Icon(color='red'), popup=row['name'])
    # popup=[folium.Popup("START", parse_html=True, max_width=100),row['start_station_name']
    # Add the marker to the map
    m.add_child(marker)



# Save the map as an HTML file
m.save('spring_evening.html')

In [104]:
#Heatmap for spring afternoon
# Create a Folium map with a base layer
m = folium.Map(location=[38.889805,-77.009056], zoom_start=11, min_opacity=0.5) #starting lat lng of washington DC

# Define the HeatMap object with the start/end locations and frequency data
heat_data = [(row['start_lat'], row['start_lng']) for index, row in df_fall_morning_filtered.iterrows()]
heat_data += [(row['end_lat'], row['end_lng']) for index, row in df_fall_morning_filtered.iterrows()]

# Create the HeatMap
heatmap = folium.plugins.HeatMap(heat_data, radius=10)

# Add the HeatMap to the Folium map
heatmap.add_to(m)



monuments = {"name": ["United States Capitol","The White House","NoMa (North of Massachusetts Avenue)",
                      "The District of Columbia Public Library","Geography of Washington, D.C. (Zero Milestone on The Ellipse)"],
             "Lat": ["38.889805","38.897957","38.900497","38.942142","38.8903"],
             "Lng": ["-77.009056","-77.036560","-77.007507","-77.025955","-77.0352"],
             }
df_monuments_geo = pd.DataFrame.from_dict(monuments, orient='columns')


for index, row in df_monuments_geo.iterrows():
    lat, lon = row["Lat"], row["Lng"]
    # Create a marker with a custom icon and pop-up
    marker = folium.Marker([lat, lon], icon=folium.Icon(color='red'), popup=row['name'])
    # popup=[folium.Popup("START", parse_html=True, max_width=100),row['start_station_name']
    # Add the marker to the map
    m.add_child(marker)

# Save the map as an HTML file
m.save('fall_morning.html')

In [105]:
#Heatmap for fall morning/afternoon
# Create a Folium map with a base layer
m = folium.Map(location=[38.889805,-77.009056], zoom_start=11, min_opacity=0.5) #starting lat lng of washington DC

# Define the HeatMap object with the start/end locations and frequency data
heat_data = [(row['start_lat'], row['start_lng']) for index, row in df_fall_morning_filtered.iterrows()]
heat_data += [(row['end_lat'], row['end_lng']) for index, row in df_fall_morning_filtered.iterrows()]

# Create the HeatMap
heatmap = folium.plugins.HeatMap(heat_data, radius=10)

# Add the HeatMap to the Folium map
heatmap.add_to(m)

monuments = {"name": ["United States Capitol","The White House","NoMa (North of Massachusetts Avenue)",
                      "The District of Columbia Public Library","Geography of Washington, D.C. (Zero Milestone on The Ellipse)"],
             "Lat": ["38.889805","38.897957","38.900497","38.942142","38.8903"],
             "Lng": ["-77.009056","-77.036560","-77.007507","-77.025955","-77.0352"],
             }
df_monuments_geo = pd.DataFrame.from_dict(monuments, orient='columns')


for index, row in df_monuments_geo.iterrows():
    lat, lon = row["Lat"], row["Lng"]
    # Create a marker with a custom icon and pop-up
    marker = folium.Marker([lat, lon], icon=folium.Icon(color='red'), popup=row['name'])
    # popup=[folium.Popup("START", parse_html=True, max_width=100),row['start_station_name']
    # Add the marker to the map
    m.add_child(marker)




# Save the map as an HTML file
m.save('fall_morning.html')